In [1]:
import numpy as np
import random

from deap import base, creator, tools, algorithms
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Sample synthetic dataset
np.random.seed(42)
X = np.random.rand(100, 5)  # Features
y = np.random.rand(100)     # Target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the evaluation function
def evaluate(individual):
    population_size, crossover_rate, mutation_rate = individual

    # Ensure valid parameter values
    population_size = int(population_size)
    crossover_rate = max(0.6, min(0.9, crossover_rate))
    mutation_rate = max(0.01, min(0.1, mutation_rate))

    # Define and train the neural network model
    nn_model = MLPRegressor(hidden_layer_sizes=(100, 50), activation="relu", solver="adam", random_state=42)
    nn_model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = nn_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return (mse,)

# Define genetic algorithm setup
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize MSE
creator.create("Individual", list, fitness=creator.FitnessMin)

# Define the genetic algorithm toolbox
toolbox = base.Toolbox()

toolbox.register("attr_pop_size", random.randint, 50, 100)
toolbox.register("attr_crossover", random.uniform, 0.6, 0.9)
toolbox.register("attr_mutation", random.uniform, 0.01, 0.1)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_pop_size, toolbox.attr_crossover, toolbox.attr_mutation), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutPolynomialBounded,
                 low=[50, 0.6, 0.01], up=[100, 0.9, 0.1], eta=1.0, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Run the genetic algorithm
population = toolbox.population(n=10)
NGEN = 20
CXPB, MUTPB = 0.7, 0.2

for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=CXPB, mutpb=MUTPB)
    fits = map(toolbox.evaluate, offspring)

    for ind, fit in zip(offspring, fits):
        ind.fitness.values = fit

    population = toolbox.select(offspring, k=len(population))

# Get the best parameters
best_ind = tools.selBest(population, k=1)[0]
print("Best Parameters:", best_ind)


Best Parameters: [91.10692625951768, 0.8730266291056911, 0.030451214986287584]
